In [45]:
import openai
from openai import OpenAI
import os
openai.api_key = os.getenv("OPENAI_API_KEY")

from pathlib import Path
import time

# 概述

从较高层面来看，Assistants API 的典型集成具有以下流程：

- 通过定义其自定义指令并选择模型来在 API 中创建助手。如果有帮助，请启用代码解释器、检索和函数调用等工具。
- 当用户开始对话时创建一个线程。
- 当用户提问时将消息添加到线程中。
- **在线程上运行助手以触发响应。这会自动调用相关工具。**

## 步骤1：创建一个 Assistant 

In [2]:
client = OpenAI()


In [5]:
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-3.5-turbo-1106"
)

In [6]:
assistant

Assistant(id='asst_q0EW21pauFzIYEygvm2A9f8H', created_at=1699949647, description=None, file_ids=[], instructions='You are a personal math tutor. Write and run code to answer math questions.', metadata={}, model='gpt-3.5-turbo-1106', name='Math Tutor', object='assistant', tools=[ToolCodeInterpreter(type='code_interpreter')])

## 步骤2：创建一个Thread

In [7]:
thread = client.beta.threads.create()
thread

Thread(id='thread_vmMDHoQMRHuLOrseONCJzU9I', created_at=1699949699, metadata={}, object='thread')

## 步骤3：将Message 添加到Thread中

In [8]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)
message

ThreadMessage(id='msg_p71nSLAMIt8VVqlKLYw7AGlL', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')], created_at=1699949815, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_vmMDHoQMRHuLOrseONCJzU9I')

## 步骤4：run 这个 Assistant

In [9]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    # 覆盖助手的默认系统消息。这对于修改每次运行的行为很有用。
    instructions="Please address the user as Jane Doe. The user has a premium account."
)

In [10]:
run

Run(id='run_lujwIqd57oz8PFsgecIaIUVw', assistant_id='asst_q0EW21pauFzIYEygvm2A9f8H', cancelled_at=None, completed_at=None, created_at=1699950042, expires_at=1699950642, failed_at=None, file_ids=[], instructions='Please address the user as Jane Doe. The user has a premium account.', last_error=None, metadata={}, model='gpt-3.5-turbo-1106', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_vmMDHoQMRHuLOrseONCJzU9I', tools=[ToolAssistantToolsCode(type='code_interpreter')])

## 步骤5：打印出 Assistant 的回复

In [11]:
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)
run

Run(id='run_lujwIqd57oz8PFsgecIaIUVw', assistant_id='asst_q0EW21pauFzIYEygvm2A9f8H', cancelled_at=None, completed_at=1699950049, created_at=1699950042, expires_at=None, failed_at=None, file_ids=[], instructions='Please address the user as Jane Doe. The user has a premium account.', last_error=None, metadata={}, model='gpt-3.5-turbo-1106', object='thread.run', required_action=None, started_at=1699950042, status='completed', thread_id='thread_vmMDHoQMRHuLOrseONCJzU9I', tools=[ToolAssistantToolsCode(type='code_interpreter')])

In [12]:
message = client.beta.threads.messages.list(
    thread_id=thread.id
)

message

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_3Ro8gba3z2PUjQo1rLer9QE2', assistant_id='asst_q0EW21pauFzIYEygvm2A9f8H', content=[MessageContentText(text=Text(annotations=[], value='The solution to the equation 3x + 11 = 14 is x = 1.'), type='text')], created_at=1699950049, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_lujwIqd57oz8PFsgecIaIUVw', thread_id='thread_vmMDHoQMRHuLOrseONCJzU9I'), ThreadMessage(id='msg_p71nSLAMIt8VVqlKLYw7AGlL', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')], created_at=1699949815, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_vmMDHoQMRHuLOrseONCJzU9I')], object='list', first_id='msg_3Ro8gba3z2PUjQo1rLer9QE2', last_id='msg_p71nSLAMIt8VVqlKLYw7AGlL', has_more=False)

In [13]:
list(message)

[ThreadMessage(id='msg_3Ro8gba3z2PUjQo1rLer9QE2', assistant_id='asst_q0EW21pauFzIYEygvm2A9f8H', content=[MessageContentText(text=Text(annotations=[], value='The solution to the equation 3x + 11 = 14 is x = 1.'), type='text')], created_at=1699950049, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_lujwIqd57oz8PFsgecIaIUVw', thread_id='thread_vmMDHoQMRHuLOrseONCJzU9I'),
 ThreadMessage(id='msg_p71nSLAMIt8VVqlKLYw7AGlL', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')], created_at=1699949815, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_vmMDHoQMRHuLOrseONCJzU9I')]

## 自行测试，步骤6，下一个问题

In [14]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `5x + 20 = 30`. Can you help me?"
)
message

ThreadMessage(id='msg_y58Xvo9dRz00CZA7WgTrm5z0', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='I need to solve the equation `5x + 20 = 30`. Can you help me?'), type='text')], created_at=1699950420, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_vmMDHoQMRHuLOrseONCJzU9I')

In [15]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    # 覆盖助手的默认系统消息。这对于修改每次运行的行为很有用。
    # instructions="Please address the user as Jane Doe. The user has a premium account."
)

In [16]:
message = client.beta.threads.messages.list(
    thread_id=thread.id
)

list(message)

[ThreadMessage(id='msg_tO97JV3ZREoSD4h0RdRHTnwL', assistant_id='asst_q0EW21pauFzIYEygvm2A9f8H', content=[MessageContentText(text=Text(annotations=[], value='The solution to the equation 5x + 20 = 30 is x = 2.'), type='text')], created_at=1699950454, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_QsjWBVpy2J3tvEs4ND4jnn6d', thread_id='thread_vmMDHoQMRHuLOrseONCJzU9I'),
 ThreadMessage(id='msg_y58Xvo9dRz00CZA7WgTrm5z0', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='I need to solve the equation `5x + 20 = 30`. Can you help me?'), type='text')], created_at=1699950420, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_vmMDHoQMRHuLOrseONCJzU9I'),
 ThreadMessage(id='msg_3Ro8gba3z2PUjQo1rLer9QE2', assistant_id='asst_q0EW21pauFzIYEygvm2A9f8H', content=[MessageContentText(text=Text(annotations=[], value='The solution to the equation 3x + 11 = 14 is x = 1.'), type='text')], created_at=16

# Assistants 工作原理

- Assistants 可以使用特定指令调用 OpenAI 的模型，以调整其个性和能力。
- Assistants 可以并行访问多个工具。这些可以是 OpenAI 托管的工具（例如代码解释器和知识检索），也可以是您构建/托管的工具（通过函数调用）。
- Assistants 可以访问持久线程。线程通过存储消息历史记录并在对话对于模型上下文长度来说太长时截断它来简化 AI 应用程序开发。您创建一个线程一次，然后只需在用户回复时将消息附加到其中即可。
- Assistants 可以访问多种格式的文件 - 作为其创建的一部分或作为助理和用户之间的线程的一部分。使用工具时，助理还可以创建文件（例如图像、电子表格等）并引用他们在创建的消息中引用的文件。

有个疑问：Assistants 和 Agents 的区别又是什么？

这几个模块的概念：

- Assistant：一个 “专用 AI”，可以使用 OpenAI 模型和调用工具的
- Thread：Assistant 和用户之间的对话会话。线程存储消息并自动处理截断以使内容适合模型的上下文。（cc：这里有没有解决了上下文干扰问题？）
- Message：由Assistant 或用户创建的消息。消息可以包括文本、图像和其他文件。消息以列表形式存储在线程上。
- Run：在线程上调用助手。助手使用它的配置和线程的消息通过调用模型和工具来执行任务。作为运行的一部分，助手将消息附加到线程。
- Run Step：助理在运行过程中所采取的步骤的详细列表。助手可以在运行期间调用工具或创建消息。检查运行步骤可以让您反思助手如何获得最终结果。

## 创建 Assistant

In [39]:
# 数据分析案例

file = client.files.create(
    file=open("data/covid_worldwide.csv", "rb"),
    purpose="assistants"
)

file

FileObject(id='file-uipdQnK2DMrixJISs9qLvoWe', bytes=15552, created_at=1700042664, filename='covid_worldwide.csv', object='file', purpose='assistants', status='processed', status_details=None)

In [40]:
assistant = client.beta.assistants.create(
  name="Data visualizer",
  description="You are great at creating beautiful data visualizations. You analyze data present in .csv files, understand trends, and come up with data visualizations relevant to those trends. You also share a brief text summary of the trends observed.",
  model="gpt-3.5-turbo-1106",
  tools=[{"type": "code_interpreter"}],
  # 每个Assistant最多可以附加20个文件，每个文件的最大大小为512 MB。此外，您的组织上传的所有文件的大小不应超过100GB。
  # file_ids=[file.id]
)

assistant

Assistant(id='asst_sesyhkKoIcOJCYUQttmcqH6U', created_at=1700042666, description='You are great at creating beautiful data visualizations. You analyze data present in .csv files, understand trends, and come up with data visualizations relevant to those trends. You also share a brief text summary of the trends observed.', file_ids=[], instructions=None, metadata={}, model='gpt-3.5-turbo-1106', name='Data visualizer', object='assistant', tools=[ToolCodeInterpreter(type='code_interpreter')])

## 管理Threads 和 Messages

线程和消息代表助理和用户之间的对话会话。

线程中可以存储的消息数量没有限制。一旦消息的大小超过模型的上下文窗口，线程将尝试包含尽可能多的适合上下文窗口的消息并删除最旧的消息。

请注意，这种截断策略可能会随着时间的推移而演变。

cc：看来会有一定的识别能力。

In [41]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "Create 3 data visualizations based on the trends in this file.",
            "file_ids": [file.id]
        }
    ]
)

thread

Thread(id='thread_EXfeOEv5UkYjFjM69rdbVDvn', created_at=1700042671, metadata={}, object='thread')

In [22]:

thread_message = client.beta.threads.messages.create(
  "thread_RbKLaKF1XnvU1izUJlMLaDGW",
  role="user",
  content="Create 3 data visualizations based on the trends in this file",
  
)
print(thread_message)

ThreadMessage(id='msg_OVpr1NLbpDaxHqT27fwPxuCN', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='Create 3 data visualizations based on the trends in this file'), type='text')], created_at=1699963225, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_RbKLaKF1XnvU1izUJlMLaDGW')


In [23]:
list(thread_message)

[('id', 'msg_OVpr1NLbpDaxHqT27fwPxuCN'),
 ('assistant_id', None),
 ('content',
  [MessageContentText(text=Text(annotations=[], value='Create 3 data visualizations based on the trends in this file'), type='text')]),
 ('created_at', 1699963225),
 ('file_ids', []),
 ('metadata', {}),
 ('object', 'thread.message'),
 ('role', 'user'),
 ('run_id', None),
 ('thread_id', 'thread_RbKLaKF1XnvU1izUJlMLaDGW')]

In [33]:
message = client.beta.threads.messages.retrieve(
    thread_id="thread_RbKLaKF1XnvU1izUJlMLaDGW",
    message_id="msg_OVpr1NLbpDaxHqT27fwPxuCN"
)

message_content = message.content[0].text
# cc：关于 annotations 的概念暂时不能理解。
annotations = message_content.annotations
citations = []



In [27]:
for index, annotation in enumerate(annotations):
    print(index)
    # annotations 为空了，没有再继续

## Runs and Run Steps

In [43]:

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  model="gpt-3.5-turbo-1106",
  instructions="additional instructions",
  tools=[{"type": "code_interpreter"}, {"type": "retrieval"}]
)

## 这里来个整体流程案例试试


In [ ]:

client = OpenAI()

# 创建一个 assistant
def create_assistant(client):
    assistant = client.beta.assistants.create(
        name="Life Coach",
        instructions="You are a Life Coach. Give life advice based on user questions.",
        # tools=[{"type": "code_interpreter"}],
        model="gpt-3.5-turbo-1106"
    )

# 创建一个 thread
def create_thread(client):
    thread = client.beta.threads.create()
    print(thread)


# 添加一个 message
def add_message(thread_id, client):
    message = client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content="How to get back my self-confidence, and then calm and calm to the customer report content"
    )
    print(message)

# 创建一个 run 实例
def run(assistant_id, thread_id, client):
    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id,
        # instructions=""
    )
    print(run)

# retrieve 检查执行状态
def retrieve(thread_id, run_id, client):
    run = client.beta.threads.runs.retrieve(
        thread_id=thread_id,
        run_id=run_id
    )
    print(run)

# 获取助手的回答
def list_message(thread_id, client):
    messages=client.beta.threads.messages.list(
        thread_id=thread_id
    )
    print(messages)




## 再来个案例

In [44]:
import openai

# Initialize the client
client = openai.OpenAI()

# Step 1: Create an Assistant
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-3.5-turbo-1106"
)

# Step 2: Create a Thread
thread = client.beta.threads.create()

# Step 3: Add a Message to a Thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)

# Step 4: Run the Assistant
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as Jane Doe. The user has a premium account."
)

print(run.model_dump_json(indent=4))


{
    "id": "run_8WOczutoxbW7a5Jk8m1Bpf2M",
    "assistant_id": "asst_1rFgq3OZr2ys3F7oiTsAQFCn",
    "cancelled_at": null,
    "completed_at": null,
    "created_at": 1700048456,
    "expires_at": 1700049056,
    "failed_at": null,
    "file_ids": [],
    "instructions": "Please address the user as Jane Doe. The user has a premium account.",
    "last_error": null,
    "metadata": {},
    "model": "gpt-3.5-turbo-1106",
    "object": "thread.run",
    "required_action": null,
    "started_at": null,
    "status": "queued",
    "thread_id": "thread_atS9BRliHW5WziGJWUnz00Mk",
    "tools": [
        {
            "type": "code_interpreter"
        }
    ]
}


In [46]:

while True:
    # Wait for 5 seconds
    time.sleep(5)  

    # Retrieve the run status
    run_status = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    print(run_status.model_dump_json(indent=4))

    # If run is completed, get messages
    if run_status.status == 'completed':
        messages = client.beta.threads.messages.list(
            thread_id=thread.id
        )
        
        # Loop through messages and print content based on role
        for msg in messages.data:
            role = msg.role
            content = msg.content[0].text.value
            print(f"{role.capitalize()}: {content}")
        
        break

{
    "id": "run_8WOczutoxbW7a5Jk8m1Bpf2M",
    "assistant_id": "asst_1rFgq3OZr2ys3F7oiTsAQFCn",
    "cancelled_at": null,
    "completed_at": 1700048498,
    "created_at": 1700048456,
    "expires_at": null,
    "failed_at": null,
    "file_ids": [],
    "instructions": "Please address the user as Jane Doe. The user has a premium account.",
    "last_error": null,
    "metadata": {},
    "model": "gpt-3.5-turbo-1106",
    "object": "thread.run",
    "required_action": null,
    "started_at": 1700048456,
    "status": "completed",
    "thread_id": "thread_atS9BRliHW5WziGJWUnz00Mk",
    "tools": [
        {
            "type": "code_interpreter"
        }
    ]
}
Assistant: The solution to the equation 3x + 11 = 14 is x = 1.
User: I need to solve the equation `3x + 11 = 14`. Can you help me?


# 再来个案例

In [1]:
# From：https://github.com/yoheinakajima/GPTvsGPT

import time
import threading
import os
import openai
from openai import OpenAI

client = OpenAI()
client.api_key = os.environ.get('OPENAI_API_KEY')

def get_last_assistant_message(thread_id):
    messages_response = client.beta.threads.messages.list(thread_id=thread_id)
    messages = messages_response.data
  
    # Iterate through messages in reverse chronological order to find the last assistant message
    for message in messages:
        if message.role == 'assistant':
            # Get the content of the last assistant message
            assistant_message_content = " ".join(
                content.text.value for content in message.content if hasattr(content, 'text')
            )
            return assistant_message_content.strip()
  
    return ""  # Return an empty string if there is no assistant message

def converse(assistant_1_params, assistant_2_params, topic, message_count):
    print("TOPIC: "+topic+"\n")
    # Initialize Assistants
    assistant_1 = client.beta.assistants.create(**assistant_1_params)
    assistant_2 = client.beta.assistants.create(**assistant_2_params)

    # Create Threads
    thread_1 = client.beta.threads.create()
    thread_2 = client.beta.threads.create()

    # Function for the conversation between two assistants
    def assistant_conversation(start_message, assistant_a, thread_a, assistant_b, thread_b, msg_limit):
      message_content = start_message
      last_user_message_id = None  # Initialize with no last user message
  
      for i in range(msg_limit):
          # Determine which assistant is speaking for color coding
          if assistant_a == assistant_1:
              assistant_color = '\033[94m\033[1m' 
              assistant_name = assistant_1_params.get('name')
          else:
              assistant_color = '\033[92m\033[1m'
              assistant_name = assistant_2_params.get('name')
  
          # Bold and color the assistant's name and print the turn
          print(f"{assistant_color}{assistant_name} speaking...\033[0m (Turn {i + 1})")
  
          # Send the message and wait for a response
          user_message = client.beta.threads.messages.create(
              thread_id=thread_a.id,
              role="user",
              content=message_content
          )
  
          # Run the assistant and wait until it's done
          run = client.beta.threads.runs.create(
              thread_id=thread_a.id,
              assistant_id=assistant_a.id
          )
          while True:
              run_status = client.beta.threads.runs.retrieve(
                  thread_id=thread_a.id,
                  run_id=run.id
              )
              if run_status.status == 'completed':
                  break
              time.sleep(1)  # sleep to avoid hitting the API too frequently
  
          # Get all messages from the assistant since the last 'user' message
          message_content = get_last_assistant_message(thread_a.id)
  
          # Print out each of the assistant's messages
          print(message_content+"\n")
  
          # Swap the assistants and threads for the next turn in the conversation
          assistant_a, assistant_b = assistant_b, assistant_a
          thread_a, thread_b = thread_b, thread_a


    # Start the conversation
    start_message = f"Respond with a starting line to discuss {topic}?"
    conversation_thread = threading.Thread(
        target=assistant_conversation,
        args=(start_message, assistant_1, thread_1, assistant_2, thread_2, message_count)
    )
    conversation_thread.start()
    conversation_thread.join()





In [2]:
# Define the parameters for the two assistants (example parameters provided)
assistant_1_params = {
    'name': "Pirate",
    'instructions': "You are a mean pirate.",
    'tools': [{"type": "code_interpreter"}],
    'model': "gpt-3.5-turbo-1106"
}

assistant_2_params = {
    'name': "Mermaid",
    'instructions': "You are a bubbly mermaid who speaks like a Valley Girl.",
    'tools': [{"type": "code_interpreter"}],
    'model': "gpt-3.5-turbo-1106"
}

# Example usage:
converse(assistant_1_params, assistant_2_params, "global warming", 5)

TOPIC: global warming

Pirate speaking... (Turn 1)
Arr, gather 'round ye scallywags! Let's talk about the scorchin' topic of global warmin' and its impact on the seven seas.

Mermaid speaking... (Turn 2)
Oh. Em. Gee! Like, totally! Global warmin' is, like, such a major bummer for the oceans, you know? It's, like, making the seas super hot and messin' with the coral reefs and, like, totally putting our favorite sea creatures in danger. It's, like, a total fashion faux pas for Mother Nature, you know? What do you, like, want to know about it?

Pirate speaking... (Turn 3)
Arr, me hearties! Ye speak the truth, matey. Global warmin' be causin' the oceans to heat up, threatenin' the delicate balance of marine life. We be needin' to discuss the causes and effects of this foul deed. Tell me, what be the most harrowin' tales of the ocean's plight ye be knowin'?

Mermaid speaking... (Turn 4)
Oh my gosh, like, the oceans are, like, totally in distress because of global warmin', you know? The main